In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import datetime
import sys
import traceback
pd.set_option('display.max_column', 100)
import time
import re
import os
import glob

In [3]:
def convert_to_epoch(df):
    depart_time = []  # Initialize the list to store departure times
    depttime=df['segmentsDepartureTimeRaw']
    for depttime in depttime :
        ts = pd.Timestamp(depttime)
        Hour = int(ts.hour)
        Minute = int(ts.minute)
        depart_time.append(f"{Hour:02d}:{Minute:02d}")  # Format the time as HH:MM
    df['depart_time'] = depart_time # Create a DataFrame from the list
        
    return df



def convert_travel_time(df):
    # Define the duration string
    duration_str = df['travelDuration']
    total_minutes=[]
    for duration in duration_str:
        # Use regular expressions to extract hours and minutes
        hours_match = re.search(r'(\d+)H', duration)
        minutes_match = re.search(r'(\d+)M', duration)
        # Initialize variables for hours and minutes
        hours = 0
        minutes = 0
        # If the matches are found, convert them to integers
        if hours_match:
            hours = int(hours_match.group(1))
            if minutes_match:
                minutes = int(minutes_match.group(1))
        # Calculate the total duration in minutes
        total_minutes.append(hours * 60 + minutes)
    
    df['travelDuration_minutes']=total_minutes
    return df


In [4]:
def process_and_save_csv(input_csv, output_parquet):
    output_path= "../data/processed/"
    df = pd.read_csv(input_csv, low_memory=False)

    df_itenary_cp = df[['searchDate', 'flightDate', 'startingAirport',
                       'destinationAirport', 'travelDuration', 'isBasicEconomy',
                       'isRefundable', 'isNonStop', 'totalFare', 'totalTravelDistance',
                       'segmentsDepartureTimeRaw', 'segmentsAirlineCode', 'segmentsCabinCode']].copy()

    df_itenary_cp['totalTravelDistance'] = df_itenary_cp['totalTravelDistance'].fillna('PT0H0M')
    df_itenary_cp['searchDate'] = pd.to_datetime(df_itenary_cp['searchDate'].str.replace('-', ''),
                                                 format='%Y%m%d', errors='coerce')
    df_itenary_cp['flightDate'] = pd.to_datetime(df_itenary_cp['flightDate'].str.replace('-', ''),
                                                 format='%Y%m%d', errors='coerce')
    df_itenary_cp['segmentsCabinCode'] = df_itenary_cp['segmentsCabinCode'].str.replace('|', ',').str.split(',').str[0]
    df_itenary_cp['segmentsDepartureTimeRaw'] = pd.to_datetime(
        df_itenary_cp['segmentsDepartureTimeRaw'].str.replace('|', ',').str.split(',').str[0],
        format='%Y-%m-%dT%H:%M:%S.%f%z', errors='coerce')

    df_itenary_cp['segmentsAirlineCode'] = df_itenary_cp['segmentsAirlineCode'].str.replace('|', ',').str.split(',').str[0]

    df_itenary_cp = df_itenary_cp.dropna()
    df_itenary_cp = convert_to_epoch(df_itenary_cp)
    df_itenary_cp = convert_travel_time(df_itenary_cp)

    df_itenary_cp = df_itenary_cp[['searchDate', 'flightDate', 'startingAirport', 'destinationAirport',
                                   'travelDuration_minutes', 'isBasicEconomy', 'isRefundable', 'isNonStop',
                                   'totalFare', 'totalTravelDistance', 'depart_time',
                                   'segmentsAirlineCode', 'segmentsCabinCode']]

    df_itenary_cp.to_parquet(output_path+output_parquet, index=False)

In [7]:
# Example usage for processing multiple CSV files
process_and_save_csv('../data/raw/DEN_full.csv','DEN_full.parquet')
process_and_save_csv('../data/raw/MIA_full.csv','MIA_full.parquet')
process_and_save_csv('../data/raw/LAX_full.csv','LAX_full.parquet')
process_and_save_csv('../data/raw/CLT_full.csv','CLT_full.parquet')
process_and_save_csv('../data/raw/BOS_full.csv','BOS_full.parquet')
process_and_save_csv('../data/raw/ATL_full.csv','ATL_full.parquet')
process_and_save_csv('../data/raw/SFO_full.csv','SFO_full.parquet')
process_and_save_csv('../data/raw/EWR_full.csv','EWR_full.parquet')
process_and_save_csv('../data/raw/ORD_full.csv','ORD_full.parquet')
process_and_save_csv('../data/raw/LGA_full.csv','LGA_full.parquet')
process_and_save_csv('../data/raw/DFW_full.csv','DFW_full.parquet')
process_and_save_csv('../data/raw/IAD_full.csv','IAD_full.parquet')
process_and_save_csv('../data/raw/OAK_full.csv','OAK_full.parquet')
process_and_save_csv('../data/raw/PHL_full.csv','PHL_full.parquet')
process_and_save_csv('../data/raw/JFK_full.csv','JFK_full.parquet')
process_and_save_csv('../data/raw/DTW_full.csv','DTW_full.parquet')

In [20]:
df_atl=pd.read_parquet('../data/processed/ATL_full.parquet')

In [21]:
df_atl['destinationAirport'].unique()

array(['BOS', 'CLT', 'DEN', 'DFW', 'DTW', 'EWR', 'IAD', 'JFK', 'LAX',
       'LGA', 'MIA', 'OAK', 'ORD', 'PHL', 'SFO'], dtype=object)

In [24]:
df_atl['flightDate'].nunique()

92

In [12]:
df_atl['segmentsCabinCode'].unique()

array(['coach', 'first', 'premium coach', 'business'], dtype=object)

In [ ]:
startingAirport
destinationAirport
flightDate


Origin airport
Destination airport
Departure date
Departure time
Cabin type ('coach', 'first', 'premium coach', 'business')

In [16]:
import os

folder_path = '../data/processed/' # Replace with the path to your folder
output_folder= "../data/processed/"
output_file = os.path.join(output_folder, 'full_file.parquet')

files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Sort the files (optional)
files.sort()
# Open the output file for writing
with open(output_file, 'wb') as outfile:
    for file in files:
        with open(os.path.join(folder_path, file), 'rb') as infile:
            # Read the contents of each file and write them to the output file
            outfile.write(infile.read())

In [19]:
# Specify the directory where your .parquet files 
input_folder = '../data/processed/' 
output_folder= "../data/processed/"

# Specify the output file where you want to save the merged data
output_file = os.path.join(output_folder, 'full_file.parquet')

# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop through the .parquet files in the input folder and merge them
for file in os.listdir(input_folder):
    if file.endswith('.parquet'):
        file_path = os.path.join(input_folder, file)
        df = pd.read_parquet(file_path)
        merged_df = pd.concat([merged_df, df])

# Save the merged data to a new .parquet file
merged_df.to_parquet(output_file, index=False)